In [1]:
print("Test!")

Test!


In [9]:
# TODO: Aduth file somehow?
import ee
import numpy as np
import geopandas
import matplotlib.pyplot as plt

# from bqplot import pyplot as plt
# from ipyleaflet import *

# ee.Authenticate();
# ee.Initialize();

import geemap
import pandas as pd


In [3]:
Map = geemap.Map(center=(40, -100), zoom=4)
Map.set_options(mapTypeId='SATELLITE');

Enter verification code:  4/1AX4XfWj4CdKvCQjuGO3gbafcD4uf8GnH9D_hlOXRjUkBiaXViif7J5uXPzk



Successfully saved authorization token.


In [4]:
shp_path = './Lhasa/Lhasa_RC_DGO2km_updated.shp';
lhasa_shp = geemap.shp_to_ee(shp_path);

/home/sky/ENS_M1_Stage_2022/.venv/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [15]:
def getCover(image, AOI, scale):
    totPixels = ee.Number(image.unmask(1).reduceRegion(
        reducer=ee.Reducer.count(),
        scale=LANDSAT_SCALE,
        geometry=AOI,
        maxPixels=1e13
    ).values().get(0));
    actPixels = ee.Number(image.reduceRegion(
        reducer=ee.Reducer.count(),
        scale=LANDSAT_SCALE,
        geometry=AOI,
        maxPixels=1e13
    ).values().get(0));
    percCover = actPixels.divide(totPixels).multiply(100).round();
    return image.set('percCover', percCover);

def maskL8sr(image):
    cloudShadowBitMask = (1 << 3);
    cloudsBitMask = (1 << 5);
    qa = image.select('QA_PIXEL');
    mask = (qa.bitwiseAnd(cloudShadowBitMask).eq(0)
    .And(qa.bitwiseAnd(cloudsBitMask).eq(0)));
    return image.updateMask(mask);

def addMNDWI(image):
    return image.addBands(srcImg=image.normalizedDifference(['B2','B5']).rename('MNDWI')); # B2=greenl B4=SWIR

def addIS_WATER_MNDWI(image):
    return image.addBands(image.expression('MNDWI >  0.0', {
            'MNDWI': image.select('MNDWI'),
            }).rename('IS_WATER_MNDWI'));
def IS_WATER_INDIC(INDIC, ImColl):
    return ImColl.select('IS_WATER_' + INDIC).map(lambda image: image.updateMask(image.neq(0)));

def process_polygon(shp, dgo=1, START_DATE='1988-04-06', END_DATE='1988-04-08'):
    filt = ee.Filter.inList('DGO_FID', [dgo]);
    AOI = shp.filter(filt);
    filtered_DGO = AOI;
    AOI_ID=f'middle_ID{1}';

    START = ee.Date(START_DATE);
    END = ee.Date(END_DATE);
    DAY_DIFF = ee.Date(END_DATE).difference(START, 'day');
    DAY_RANGE = ee.List.sequence(0,DAY_DIFF.subtract(1),1).map(lambda day: START.advance(day,'day'));

    #  /* Working resolution, minimum 30 meters, read
    #                          https://developers.google.com/earth-engine/guides/scale
    #                          to learn how scales/images pyramids work in GEE */

    eightCo = True;      # // eightConnected boolean for vectorization
    nCo = (eightCo+1)*4; #  // NConnected integer for file exports

    #/* int or float, multiplied by LANDSAT_SCALE, becomes the tolerance 
    #                                  of the simplification of the water polygons */

    # Visulization parameters :

    NDWI_MIN = -0.3;
    NDWI_MAX = 0.3;

    NDVI_MIN = -0.5;
    NDVI_MAX = 0.5;

    CLOUD_PAL = ['gray', 'white'];
    NDWI_PAL = ['black','white','blue'];
    NDVI_PAL = ['black','white','green'];

    LANDSAT_FULL = (ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
        .select(['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5','SR_B6','SR_B7','QA_PIXEL'])
        .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL'])));
    LANDSAT_FULL = (LANDSAT_FULL
                        .merge(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2').select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5','SR_B7','QA_PIXEL']))
                        .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL']))
                        .merge(ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5','SR_B7','QA_PIXEL']))
                        .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL']))
                        .merge(ee.ImageCollection('LANDSAT/LT04/C02/T1_L2').select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4','SR_B5','SR_B7','QA_PIXEL']))
                        .map(lambda image: image.rename(['B1', 'B2', 'B3', 'B4','B5','B7','QA_PIXEL']))   
                    );

    LANDSAT = (LANDSAT_FULL
        .filterDate(START_DATE, END_DATE)
        .filter(ee.Filter.lte('CLOUD_COVER', CLOUD_FILTER))
        .map(maskL8sr)
        .filterBounds(AOI));

    # LANDSAT = LANDSAT.map(lambda image: getCover(image, AOI, LANDSAT_SCALE));
    # LANDSAT = LANDSAT.filterMetadata('percCover', 'greater_than', 98);
    LANDSAT = LANDSAT.map(addMNDWI);

    LANDSAT = LANDSAT.map(addIS_WATER_MNDWI);

    IS_WATER_MNDWI = IS_WATER_INDIC('MNDWI', LANDSAT);
    return AOI, LANDSAT, IS_WATER_MNDWI;

MIN_PIX = 2; # // To filter small water polygons
CLOUD_FILTER = 80;
LANDSAT_SCALE = 30;
SIMPLIFY_TOLERANCE_N = 2;
AOI, LANDSAT, IS_WATER_MNDWI = process_polygon(shp=lhasa_shp, dgo=1);

Map.addLayer(LANDSAT.first(), {"bands": ['B3', 'B2', 'B1'], "min": 9600, "max": 16677}, 'RGB');
Map.addLayer(lhasa_shp, {}, 'Lhasa River SHP');
Map.addLayer(IS_WATER_MNDWI, {'palette': 'Reds'}, 'water MNDWI', True);
Map.zoom = 11
Map.center_object(lhasa_shp);

In [13]:
Map

Map(bottom=216718.71421813965, center=[29.651569731000073, 91.3219064102816], controls=(ZoomControl(options=['…

In [18]:
def VECTORIZE_FOR_AOI(AOI):
    def VECTORIZE(image):
        DAY_VECTORS = image.reduceToVectors(**{
            "geometry" : AOI,
            "scale" : LANDSAT_SCALE,
            "eightConnected" : True,
            "maxPixels" : 1e12,
            "geometryType" : 'polygon'
        });
        return DAY_VECTORS.set('DATE_ACQUIRED',image.get('DATE_ACQUIRED'));
    return VECTORIZE;

# Add area info to filter out isolated pixels
def ADDPROPS_FEAT(feature): 
    return feature.set({'AREA' : feature.area(LANDSAT_SCALE)});

def WRAP_COL_FOR_AOI(AOI):
    def WRAP_COL(collection):
        collection = (ee.FeatureCollection(collection).map(ADDPROPS_FEAT)
            .filterMetadata('AREA', 'greater_than', MIN_PIX * LANDSAT_SCALE * LANDSAT_SCALE));
        time_start = collection.get('DATE_ACQUIRED');
        collection.map(lambda feature: feature.set({'DATE_ACQUIRED' : time_start}));
        geom = collection.geometry();
        geom_smooth = collection.geometry().simplify(**{'maxError' : SIMPLIFY_TOLERANCE_N*LANDSAT_SCALE});
        polygon = AOI.geometry(); #this is the polygon of the AOI/DGO
        polygon_smooth = AOI.geometry().simplify(**{'maxError' : SIMPLIFY_TOLERANCE_N*LANDSAT_SCALE})

        return ee.Feature(geom, {
            'TOTAL_PERIM' : geom.perimeter(LANDSAT_SCALE),
            'AREA' : geom.area(LANDSAT_SCALE),
            'TOTAL_PERIM_SMOOTH' : geom_smooth.perimeter(LANDSAT_SCALE),
            'AREA_SMOOTH' : geom_smooth.area(LANDSAT_SCALE),
            'AREA_POLYGON' : polygon_smooth.area(LANDSAT_SCALE), # this is the polygon of the AOI/DGO
            'SENSING_TIME' : collection.get('DATE_ACQUIRED')
        });
    return WRAP_COL;

def WATER_VEC(AOI, ImColl):
    WVEC = ImColl.map(VECTORIZE_FOR_AOI(AOI));
    return ee.FeatureCollection(WVEC.map(WRAP_COL_FOR_AOI(AOI)));


RIVER_PROPERTIES = [];
# TODO: Read DGO Count from shp 
for i in range(1, 78):
    # TODO: Print using ipython notify (progress bar...)
    print("DGO:", i, "/", 78);
    AOI, LANDSAT, IS_WATER_MNDWI = process_polygon(shp=lhasa_shp, dgo=i, START_DATE='1980-01-01', END_DATE='2100-01-01');
    ISWV_MNDWI = WATER_VEC(AOI, IS_WATER_MNDWI);
    info = ISWV_MNDWI.getInfo(); # PROCESS AND PULL RESULT
    property_list = map(lambda x: {
            "ID": i,
            "Tile_ID": x['id'],
            **x['properties']
        }, info['features']);
    # Map over properties (calculate Braiding Index e.t.c.)
    RIVER_PROPERTIES += list(property_list);

pd.DataFrame(RIVER_PROPERTIES).to_csv("test.csv");
print("Done!");

DGO: 1 / 78
DGO: 2 / 78
DGO: 3 / 78
DGO: 4 / 78
DGO: 5 / 78
DGO: 6 / 78
DGO: 7 / 78
DGO: 8 / 78
DGO: 9 / 78
DGO: 10 / 78
DGO: 11 / 78
DGO: 12 / 78
DGO: 13 / 78
DGO: 14 / 78
DGO: 15 / 78
DGO: 16 / 78


KeyboardInterrupt: 

In [8]:
df = pd.read_csv("test.csv")
df['BRAIDING_INDEX'] = (df['TOTAL_PERIM_SMOOTH']/2)/2000; # Calculate braiding index
df

FileNotFoundError: [Errno 2] No such file or directory: 'test.csv'

In [83]:
df_c = df.copy();

fig, axs = plt.subplots(9, 9, figsize=(24, 18), constrained_layout=True);
axs = axs.flatten();
for k, v in enumerate(range(1, 77)):
    df_d = df_c[df_c["ID"] == v];
    axs[k].set_title(v);
    axs[k].plot(df_d["SENSING_TIME"], df_d["BRAIDING_INDEX"], '+');

In [ ]:
# For call:
# Maybe try to get things working for Nelly...
# Python 3.10: https://github.com/google/earthengine-api/issues/181
# Python 3.9: Works
# Modules:
# numpy, pandas, matplotlib
# For Windows: pipwin gdal, fiona
# geemap, geopandas, ee
# TODO List:
# Fix laptop (DONE)
# Fix ownCloud (DONE)
# Email Eric about structure/limits of M1 Report (DONE) [reponse]


# Barbara:
# Look into making algorithm faster (doing all 78 in one pass?) [Currently it takes 1H to run...]
    # MAYBE I can run multiple .info() in seperate threads(??) (does google allow multiple requests like this?)
    # Cannot map over DGOs in a shape in a single command...
    # PERHAPS modify process_polygon (make it general) move it out of the loop!
    # Do process_polygon on ALL DGOs first - we can REUSE the same water map accordingly

# Try to make mosaic to avoid overlapping values. (Look at google earth api)
# https://developers.google.com/earth-engine/guides/ic_composite_mosaic

# Run again to extract SENTINEL data (from high-res sattelites)
  # REF: Barbara's email
# [Comparison between Sentinel & Landsat Data]
# Support for downloading tiles from meta information --> to use for GIS fieldwork

# Nelly/Patrice:
# Document installation procedure
# Reorganise repository 
# Write README.md

# For Monday:
# - extract the 4 channel images for the 200 timeslots and share them with me [finer scale analysis?]
# - extracted cvs. + processed data with l1 optimisation method for 3 differet level of cloud threshold
